# Getting Even - Coding Challenge - F#

## Description

A coding challenge:
1. There is a 4x6 grid.
2. 18 spots must be filled in.
3. The number of spots in each row &amp; column must be even.
4. Print out a 4x6 grid (or equivalent) showing the result.

## Solution Discussion
The aim here is to create a functional solution in F#, for an arbitrary MxN grid.

As an additional "trick", optimise for the case where the number of spots is > (M+N)/2 by instead solving for "MxN - number of spots" and then inverting the result.

## Setup

In [4]:
let Mrows: int = 4 // must be even and > 0
let Ncols: int = 6 // must be even and > 0
let NumSpots: int = 18 // must be even and >= 4 and < Mrows*Ncols

In [5]:
let isEven x = (x % 2) = 0

let boolToInt b =
    match b with
    | true -> 1
    | false -> 0

let invert (b:Boolean) = not b

let check (testValue: Boolean) (checkMessage: String) =
    if (not testValue) then raise (Exception checkMessage)

let checkGridParams (mrows:int) (ncols:int): unit =
    check ((mrows > 0) && isEven(mrows)) (sprintf "invalid mrows: %d" mrows)
    check ((ncols > 0) && isEven(ncols)) (sprintf "invalid ncols: %d" ncols)

let checkProblemParams (mrows:int) (ncols:int) (numSpots:int): unit =
    checkGridParams mrows ncols
    let maxSpots = mrows*ncols
    check ((numSpots >= 4) && (numSpots < maxSpots) && isEven(numSpots)) (sprintf "invalid numSpots: %d" numSpots)

In [43]:
let MaxSpots = Mrows * Ncols
let OptimalNumSpots = Math.Min(NumSpots, MaxSpots - NumSpots)
printfn "#spots = %d; optimal #spots = %d" NumSpots OptimalNumSpots

#spots = 18; optimal #spots = 6


## Grid Implementation

We need to support the concept of a grid of Boolean "spots" that we can set (to true) or reset (to false).

In [20]:
let updateList<'a> (newValue:'a) (index:int) (aList: 'a list): 'a list =
    List.concat [aList |> List.take (index); [newValue] ; aList |> List.skip (index+1)]

let rec ListToReversedCols<'a> (rows: int) (acc: 'a list list) (isSpot: 'a list): 'a list list =
    match isSpot with
    | [] -> acc
    | spotList ->
        let nextCol = isSpot |> List.take rows
        ListToReversedCols rows (nextCol::acc) (isSpot |> List.skip rows)

let SpotsToString (isSpot: Boolean list): String =
    isSpot |> List.map (fun spot -> if (spot) then "X" else "_") |> String.concat " "

type Grid =
    { Rows: int; Cols: int; IsSpot: Boolean list } // IsSpot is column-by-column list of Boolean values

    member this.CheckValid(): unit =
        assert(this.IsSpot.Length = this.Rows*this.Cols)
    
    member this.GetColumns(): Boolean list list =
        let reversedResult: Boolean list list = ListToReversedCols this.Rows [[]] this.IsSpot
        reversedResult |> List.rev |> List.tail // reverse the results list and remove the initial empty list

    member this.GetRows(): Boolean list list =
        let cols = this.GetColumns()
        seq{
            for idx in 0 .. (this.Rows-1) do
                yield cols |> List.map (fun col -> col.[idx])
         } |> Seq.toList
    
    member this.ToGridString(): String =
        let rows = this.GetRows()
        rows |> List.map SpotsToString |> String.concat "\n"

    member this.UpdateIndex (index: int) (newValue:Boolean): Grid =
        let result = {this with Grid.IsSpot = (updateList newValue index this.IsSpot)}
        result.CheckValid()
        result
    
    member this.Update (row:int) (col:int) (newValue:Boolean): Grid =
        this.UpdateIndex (col*this.Rows+row) newValue
            
    member this.Set (row:int) (col:int): Grid = this.Update row col true

    member this.Reset (row:int) (col:int): Grid = this.Update row col false

    member this.SetIndices (indices: int list): Grid =
        match indices with
        | [] -> this
        | head::tail -> (this.UpdateIndex head true).SetIndices(tail)
    
    member this.IsValidSolution (): Boolean = // check if every row and column has an even number of "spots"
        let columnsValid = (this.GetColumns() |> List.map (fun column -> column |> List.map boolToInt |> List.sum |> isEven) |> List.filter (fun bool -> not bool) |> List.length) = 0
        if (columnsValid)
        then
            let rowsValid = (this.GetRows() |> List.map (fun column -> column |> List.map boolToInt |> List.sum |> isEven) |> List.filter (fun bool -> not bool) |> List.length) = 0
            rowsValid
        else false
    
    member this.Invert: Grid = {this with Grid.IsSpot = this.IsSpot |> List.map invert}

    static member NewFalseGrid (mrows:int) (ncols:int): Grid =
        let result = {Rows = mrows; Cols = ncols; IsSpot = (List.init (mrows*ncols) (fun _ -> false))}
        result.CheckValid()
        result

Let's just test some of this code.

In [21]:
updateList false 1 [true;true;true]

[ True, False, True ]

In [22]:
let grid = (Grid.NewFalseGrid 3 5).SetIndices([0;2;4;6;8;10;12;14])
grid.CheckValid()
grid.ToGridString()

X _ X _ X
_ X _ X _
X _ X _ X

In [23]:
(grid.Set 0 1).ToGridString()

X X X _ X
_ X _ X _
X _ X _ X

## Solution Logic

```nextIndices``` incremements a set of indices, starting by trying to increment the left-most index.
If it can't increment it (already at the maximum), it sets that index to zero and tries to increment the index to its right, and so on.
If it can't increase any index in a valid way, it returns 'None'.

Importantly, **none of the indices are allowed to be the same**.

In [24]:
let rec nextIndices (maxIndex: int) (indices: int list): (int list) option =
    match indices with
    | [] -> None
    | _ ->
        let result =
            match indices.Head with
            | idx when idx = maxIndex -> // can't increment the head of indices
                let newTail = nextIndices maxIndex indices.Tail
                match newTail with
                | None -> None
                | Some(tail) -> Some(0::tail)
            | idx -> // can increment the head of indices
                Some((idx + 1)::(indices.Tail))
        if (result.IsNone)
        then None
        elif ((result.Value |> Set.ofList).Count < indices.Length)
        then nextIndices maxIndex result.Value // if any two indices are the same, keep iterating
        else result

In [25]:
nextIndices 3 [0;0;0;0]

Value,"[ 3, 2, 1, 0 ]"


In [26]:
nextIndices 3 [3;2;1;0]

Value,"[ 2, 3, 1, 0 ]"


In [29]:
nextIndices 3 [3;3;3;3] // Jupyter shows the result as <null> but it's really None.

<null>

Now we look for a solution iteratively, by modifying the index locations of the "spots" and looking for the first spot configuration that is a solution.

In [30]:
let solveGrid (mrows:int) (ncols:int) (numSpots:int): Boolean*Grid = // "(success?, success grid)
    checkProblemParams mrows ncols numSpots
    let startGrid = Grid.NewFalseGrid mrows ncols
    let rec iterateSolution (maxIndex:int) (indices:int list): Boolean*Grid =
        let newIndices = nextIndices maxIndex indices
        match newIndices with
        | None -> (false, startGrid)
        | Some(indices) ->
            let newGrid = startGrid.SetIndices newIndices.Value
            if (newGrid.IsValidSolution())
            then (true, newGrid)
            else iterateSolution maxIndex newIndices.Value
    iterateSolution (mrows*ncols-1) (List.replicate numSpots 0)

## Run Solution

In [31]:
let success, grid = solveGrid Mrows Ncols OptimalNumSpots
if (success)
then
    if (OptimalNumSpots = NumSpots)
    then printfn "%A" (grid.ToGridString())
    else printfn "%A" (grid.Invert.ToGridString())
else printfn "FAILED: no solution found"

Error: input.fsx (1,43)-(1,58) typecheck error This expression was expected to have type
    'int'    
but here has type
    'int * int -> int * int'    
input.fsx (4,9)-(4,24) typecheck error The type '(int * int -> int * int)' does not support the 'equality' constraint because it is a function type
input.fsx (4,27)-(4,35) typecheck error This expression was expected to have type
    'int * int -> int * int'    
but here has type
    'int'    

<span style="color:red;font-weight:bold">!!!! The above solution is the wrong shape !!!!</span>

In [ ]:
grid

In [ ]:
grid.GetColumns()

In [ ]:
grid.GetRows()